In [ ]:
# import numpy as np 
import pandas as pd
from sqlalchemy import create_engine
from project_secrets import REDSHIFT_CONNECTION_STRING, AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
red_engine = create_engine(REDSHIFT_CONNECTION_STRING)

### Lake !

In [ ]:
# Datalake

collab_sessions_dl = """

unload($$

select wt.session_id,
  count(distinct wt.user_id) qq
  from datalake.collab_time_v0_ext wt
  group by 1
  having qq > 1

$$) to 's3://mural-data-lake-prod/data/processed/data-science/collab_sessions/collab_sessions'
parquet
iam_role 'arn:aws:iam::882722260434:role/redshiftSpectrum' allowoverwrite;

"""

In [ ]:
red_engine.execute(collab_sessions_dl)

### Warehouse !

In [ ]:
# Warehouse

collab_sessions_wh = """

drop table if exists datascience_stage.collab_sessions;
create table datascience_stage.collab_sessions as

    (select wt.session_id,
        count(distinct wt.user_id) qq
    from datalake.collab_time_v0_ext wt
    group by 1
    having qq > 1);

grant select on datascience_stage.collab_sessions to periscope;

"""

In [ ]:
red_engine.execute(collab_sessions_wh)

## create from redshift to S3

In [ ]:
query_table_s3 = """
CREATE TABLE sarasa
WITH (
      external_location = 's3://my_athena_results/my_orc_stas_table/',
      format = 'PARQUET')
AS SELECT * 
FROM old_table;

"""

In [ ]:
red_engine.execute(query_table_s3)

## read from redshift

In [ ]:
query_test = """
select top 50
* 
from public.memberv2 

"""

In [ ]:
df_test = pd.read_sql_query(query_test, red_engine)
df_test.head()

# read files from S3

In [ ]:
# LOAD parquet
import pyarrow.parquet as pq
import s3fs
fs = s3fs.S3FileSystem(key=AWS_ACCESS_KEY_ID, secret=AWS_SECRET_ACCESS_KEY)

path = 's3://mural-data-lake-prod/data/processed/data-science/hot_spots_from_bad_fps_v0/added_connector_count_14days_v0'
df_tmp = pq.ParquetDataset(path, filesystem=fs)
df_tmp = df_tmp.read_pandas()
df_tmp = df_tmp.to_pandas()


In [ ]:
df_tmp.head(4)

In [ ]:
#CSV
# levantar varios files de una misma ruta 
# !pip3 install "dask[dataframe]"
import dask.dataframe as dd

path = "s3://mural-data-lake-prod/data/processed/data-science/temp/*.csv"
df_from_cvs = dd.read_csv(path,
                          sep=",", 
                          storage_options={"key":AWS_ACCESS_KEY_ID, 
                                           "secret":AWS_SECRET_ACCESS_KEY}) 
df_from_cvs = df_from_cvs.compute()

In [ ]:
df_from_cvs.head(3)

# write files on S3
### Save file csv

In [ ]:

import boto3
s3 = boto3.client('s3', 
                  aws_access_key_id=AWS_ACCESS_KEY_ID, 
                  aws_secret_access_key=AWS_SECRET_ACCESS_KEY)


csv_file_name = "csv_file_name.csv"
bucket_name = 'mural-data-lake-prod'
path_name = 'data/processed/data-science/temp/tmp.csv'

df_to_upload.to_csv(csv_file_name)

files=['interes_item_df']
for i in files:
    s3.upload_file(csv_file_name,bucket_name,path_name)


### Save file parquet

In [ ]:
import s3fs
fs = s3fs.S3FileSystem(key=AWS_ACCESS_KEY_ID, secret=AWS_SECRET_ACCESS_KEY)

filename= "s3://mural-data-lake-prod/data/processed/data-science/temp/file_tmp.parquet"
df_to_upload.to_parquet(fs.open(filename,'wb'))